<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# RAG + Wikipedia (for MMLU)

In [3]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()
file = open("info/path.txt", "r")
data_path = file.read()
file.close()
file = open("info/user.txt", "r") 
user = file.read()
file.close()

In [4]:
# read samples.csv file
import pandas as pd
import os

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers


## 1. build DB (using Wikipida)

### what is the main topic & specific topic of query

In [5]:
prompts, answers = read_data(os.path.join(data_path, 'mmlu_pro.csv'))

In [6]:
testdata = pd.read_csv(data_path+'mmlu_pro.csv')
testdata

,prompts,answers
0,QUESTION0) The symmetric group $S_n$ has $\n\f...,(A)
1,QUESTION1) Let V be the set of all real polyno...,(H)
2,QUESTION2) Let A be the set of all ordered pai...,(E)
3,QUESTION3) A tank initially contains a salt so...,(I)
4,QUESTION4) A total of 30 players will play bas...,(B)
...,...,...
12097,QUESTION12252) A hot mild steel rod is placed ...,(J)
12098,QUESTION12253) The cost of making the correct ...,(H)
12099,QUESTION12254) Consider the evaporation of liq...,(F)
12100,"QUESTION12255) Air (100°F, 1atm) is flowing at...",(I)


In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    Answer according to the conditions.
    1. Return the broad topic and specific topic for the given [Sentence].
    2. Respond with ONLY the two words that correspond to [Answer].
    3. The answer format is AS FOLLOW.
    4. Do not output any other characters.

    Q : Return the topic of the given sentence as two keyword, one as a broad topic, and one as a specific term.
        [Sentence] : The _________ of a number is its logarithm to the base of the mathematical constant e.
    [Answer] math,logarithm

    Q : Return the topic of the given sentence as two keyword, one as a broad topic, and one as a specific term.
        [Sentence] : Who is the person making the argument that “reason seems to be able to prove that free will cannot be a causally efficacious part of the world (because all of nature is deterministic) and yet that it must be such a cause”?
    [Answer] philosophy,Immanuel Kant

    Q : Return the topic of the given sentence as two keyword, one as a broad topic, and one as a specific term.        
        [Sentence] : {question}
    [Answer]

    '''

)
chain_2shot = prompt_template | llm


In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    Answer according to the conditions.
    1. Return the broad topic and specific topic for the given [Sentence].
    2. Respond with ONLY the two words that correspond to [Answer].
    3. The answer format is AS FOLLOW.
    4. Do not output any other characters.

    [Sentence] Who is the person making the argument that “reason seems to be able to prove that free will cannot be a causally efficacious part of the world (because all of nature is deterministic) and yet that it must be such a cause”?
    [Answer] philosophy,Immanuel Kant

    [Sentence] {question}
    [Answer] 


    '''

)
chain_1shot = prompt_template | llm


In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    Return the topic of the given sentence ONLY as two word, one as a broad topic, and one as a specific topic.
    Return TWO word like (a broad topic ,a specific topic).

    Sentence : {question}
    topic: 

    Do not output any other characters.
    
    '''

)
chain_zero = prompt_template | llm


In [ ]:
querys = testdata.prompts
responses = []
for query in querys[:30] :
    response = chain_1shot.invoke({"question": query.partition('(A)')[0]}) # 선지 전까지 받아오기
    responses.append(response.content)

In [ ]:
responses

['mathematics,group theory',
 'mathematics,calculus',
 'mathematics,combinatorics',
 '\n    \n    [Answer] physics,engineering',
 'math,division',
 'nasal cavity',
 'embryology,hyoid bone',
 'medical,catheter',
 'medicine,cannulation',
 'virology,parvoviruses',
 'astronomy,optics',
 'Astrophysics, Oort Cloud',
 'astronomy,refracting telescope',
 'physics,optics',
 'physics,electricity',
 'social media, corporate social responsibility, social media, sustainability goals',
 'ethics,management',
 'democratic,inclusive',
 'Email',
 'purchasing department',
 'biology,Arthropods',
 'biology,genetics',
 'biology,enzyme',
 'biochemical synthesis',
 'biology,cell division',
 'chemistry,spectroscopy',
 'chemistry,hydrides',
 'organic chemistry,acid anhydride',
 'chemistry,acid',
 'chemistry,acid-base equilibrium']

### 멍청한 solar를 위해 프롬프트를 나누다.

In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    Answer according to the conditions.
    1. Return the broad topic for the given [Sentence] as is.
    2. Respond with ONLY One Word that correspond to [Answer].
    3. The answer format is AS FOLLOW.
    4. Do NOT output ANY OTHER characters.

    [Sentence] The _________ of a number is its logarithm to the base of the mathematical constant e.
    [Answer] math,logarithm

    [Sentence] {question}
    [Answer] 

    '''

)
chain_1shott = prompt_template | llm

querys = testdata.prompts
responses = []
for query in querys[:30] :
    response = chain_1shott.invoke({"question": query.partition('(A)')[0]}) # 선지 전까지 받아오기
    responses.append(response.content)

responses

NameError: name 'api_key' is not defined

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    Answer according to the conditions.
    1. Return the SPECIFIC topic for the given [Sentence].
    2. Respond with ONLY One Word that correspond to [Answer].
    3. The answer format is AS FOLLOW.
    4. Do NOT output ANY OTHER characters.

    [Sentence] Who is the person making the argument that “reason seems to be able to prove that free will cannot be a causally efficacious part of the world (because all of nature is deterministic) and yet that it must be such a cause”?
    [Answer] Immanuel Kant

    [Sentence] {question}
    [Answer] 
    

    '''

)
chain_1shott = prompt_template | llm

querys = testdata.prompts
responses = []
for query in querys[:30] :
    response = chain_1shott.invoke({"question": query.partition('(A)')[0]}) # 선지 전까지 받아오기
    responses.append(response.content)

responses

NameError: name 'api_key' is not defined

#### 한 단어로 요약시켜버리기 ㅋㅋ

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    Retrun a main word from the given sentence.

    [Sentence] Who is the person making the argument that “reason seems to be able to prove that free will cannot be a causally efficacious part of the world and yet that it must be such a cause”?
    [Answer] philosophy

    [Sentence] {question}
    [Answer] 
    

    Return with only A word!!!!!!!!!!!!!!!!!!!!!!!!!
    
    '''

)

chain_summ = prompt_template | llm

querys = testdata.prompts
responses = []
for query in querys[:30] :
    response = chain_summ.invoke({"question": query.partition('(A)')[0]}) # 선지 전까지 받아오기
    responses.append(response.content)

responses

['To solve this task, we can follow these steps:\n\n1. Identify the main word in the given sentence.\n2. Remove any additional words or phrases that are not part of the main word.\n3. Return the main word as the answer.\n\nHere\'s the Python code to accomplish this:\n\n```python\n# Step 4: Given the sentence, identify the main word and remove any additional words or phrases.\nsentence = "The symmetric group $S_n$ has $\\\\factorial{n}$ elements, hence it is not true that $S_{10}$ has 10 elements. Find the characteristic of the ring 2Z."\n\n# Step 5: Remove any additional words or phrases that are not part of the main word.\nmain_word = "characteristic"\n\n# Step 6: Return the main word as the answer.\nmain_word\n```',
 'The main word from the given sentence is "mathematics".',
 'To solve this problem, we can use a simple approach:\n\n1. Split the sentence into words.\n2. Get the first word.\n3. Return the first word.\n\nHere is the Python code to solve this problem:\n\n```python\ndef m

### zer-shot으로 노선 개같이 변경

In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    Answer according to the conditions.
    1. Return the broad topic for the given [Sentence].
    2. Respond with ONLY One Word that correspond to [Answer].
    3. Do NOT output ANY OTHER characters.

    [Sentence] {question}
    The broad topic is

    '''

)
chain_1shott = prompt_template | llm

querys = testdata.prompts
broad_topics = []
for query in querys[:30] :
    response = chain_1shott.invoke({"question": query.partition('(A)')[0]}) # 선지 전까지 받아오기
    broad_topics.append(response.content)

broad_topics

['Algebra',
 'Mathematics',
 'Mathematics.',
 'Chemical_Engineering',
 'Mathematics',
 'Anatomy',
 'Anatomy',
 'Medical devices',
 'Medicine',
 'Viruses',
 'Physics',
 'Astronomy',
 'Optics',
 'Physics',
 'Electricity',
 'Sustainability',
 'Ethics',
 'Management.',
 'Advertising',
 'purchasing',
 'Arthropods',
 'Genetics',
 'Mutation',
 'Biology',
 'Biology',
 'Spectroscopy',
 'Chemistry',
 'Chemistry',
 'Chemistry',
 '\nAcid-Base Reaction']

### link to Wikipedia

답변으로 받아온 두 개의 토픽 -> wikipedia 가져오기

In [7]:
import wikipediaapi

# user는 내 아이디 같은 거
wiki_wiki = wikipediaapi.Wikipedia(f'{user}', 'en')

In [8]:
responses

['Algebra',
 'calculus',
 'Algebra.',
 'CHEMISTRY',
 'Division.',
 'sella turcica',
 'first pharyngeal arch',
 'Anatomy',
 'Medicine',
 'DNA',
 'Magnification',
 'Oort Cloud',
 'Telescope',
 'PHYSICS',
 'Power',
 '보이콧',
 'Ethics.',
 'Democratic.',
 'Email',
 'buyers',
 'Insects',
 'homozygous',
 'Mutation.',
 'Transfection.',
 'Mitosis.',
 'Spectroscopy',
 'Silicon.',
 'Carboxylic',
 'acid',
 'Acid-Base Equilibrium']

In [9]:
for broad in responses :
    wiki_broad = wiki_wiki.page(f'{broad}')
    print(" Broad Topic Page - Exists: %s" % wiki_broad.exists())

 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: False
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: False
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: False
 Broad Topic Page - Exists: False
 Broad Topic Page - Exists: False
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: False
 Broad Topic Page - Exists: False
 Broad Topic Page - Exists: False
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: False
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: True
 Broad Topic Page - Exists: False


In [ ]:
# 해당 topic에 해당하는 Wikipedia가 있는지 확인하기
wiki_broad = wiki_wiki.page(f'{broad}')
print(" Broad Topic Page - Exists: %s" % wiki_broad.exists())

wiki_speci = wiki_wiki.page(f'{specific}')
print(" Specific Topic Page - Exists: %s" % wiki_speci.exists())


Page - Exists: True


In [ ]:
# topicd에 해당하는 Wikipedia의 title과 본문 길이 출력
print("Page - Title: %s" % wiki_broad.title)
print(len(wiki_broad.text))

# topic에 해당하는 Wikipedia의 title과 본문 길이 출력
print("Page - Title: %s" % wiki_speci.title)
print(len(wiki_speci.text))


Page - Title: Hip hop music
102199


## 2. Text Split (chunking Wikipedia)

In [ ]:
from langchain_text_splitters import (Language, RecursiveCharacterTextSplitter,)
from langchain.schema import Document

# 받아온 Wikipedia를 split함수에 적용할 수 있도록 변환
docs = wiki_query.text
docs = Document(page_content=docs)

# 2. Split **** (hyper param)
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=10000, chunk_overlap=2500, language=Language.HTML
)
splits = text_splitter.split_documents([docs])
print("Splits:", len(splits))

Splits: 14


In [ ]:
print(splits[1])

In [ ]:
# chunking된 context 확인하기
splits[1].page_content

## 3. query-context matching

In [ ]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

## 4. Prompt engineering (for answering)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    Please provide most correct answer from the following context.
    If the answer is not present in the context, please Answer "The information is NOT present in the context."
    
    The Answer format is as follow (do not explain) : 
    Answer: (D) keyword.
    ---
    
    Question: {question}
    Context: {context}
    Answer :
    ---
        
    '''

)
chain = prompt_template | llm


In [ ]:
# 3+4. 하나로 통합
# chunked Wikipedia -> embedding 

import numpy as np

chunk = []
for index in range(0,len(splits)) : # context 받아오기
    chunk.append(splits[index].page_content)

responses = []
for prompt in prompts[:1] : # 질문 받아오기 
    embedded_query = query_embeddings.embed_query(prompt.split('\n')[0])
    embedded_documents = passage_embeddings.embed_documents(chunk)

    # 질문(embedded_query):
    similarity = np.array(embedded_query) @ np.array(embedded_documents).T

    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

    # QA
    response = chain.invoke({"question": prompt, "context": sorted_idx[0]})
    responses.append(response.content)

In [ ]:
for i in responses:
    print(i)
    print('-')


To answer the question, I will need more information about what specific aspect of the COVID-19 pandemic you would like me to discuss. Please provide more details or specify the topic you would like me to cover.
-


## Check Accuracy

In [ ]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [ ]:
# print accuracy

cnt = 0

for answer, response in zip(answers, responses):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")


    if generated_answer == None:
        continue
    if generated_answer in answer:
        cnt += 1

print()
print(f"acc: {(cnt/len(answers))*100}%")

----------
To answer the question, I will need more information about what specific aspect of the COVID-19 pandemic you would like me to discuss. Please provide more details or specify the topic you would like me to cover.
generated answer: I, answer: (A)

acc: 0.0%
